In [1]:
import torch

# Check if a GPU is available
gpu_available = torch.cuda.is_available()

print(f"Is GPU available? {gpu_available}")

Is GPU available? True


In [2]:
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 22.1 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 30.6 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]


In [3]:
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1"
# !pip install --upgrade -qqq uv
!pip install -qqq --upgrade unsloth vllm
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 5.2 MB/s  0:00:02eta 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.8 MB/s  0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.2
    Uninstalling transformers-4.56.2:╺━━━━━━━━━━━━━━━━━━━ 1/2 [transformers]
      Successfully uninstalled transformers-4.56.2━━━━━━━━━━━━━━━━ 1/2 [transformers]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers] [transformers]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.23.0 requires transformers>=4.56.1, but you have transformers 4.55.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 8.5 M

In [4]:
from unsloth import FastLanguageModel
import torch
import pandas as pd
import numpy as np
from datasets import Dataset
import re

# ============================================================================
# STEP 1: Load Model
# ============================================================================
max_seq_length = 1024
lora_rank = 32

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Base",
    max_seq_length = max_seq_length,
    load_in_4bit = False,
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.9,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 10-08 23:38:01 [__init__.py:216] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 10-08 23:38:06 [vllm_utils.py:689] Unsloth: Patching vLLM v1 graph capture
Unsloth: Could not patch vLLM V0 graph capture: No module named 'vllm.worker.model_runner'
==((====))==  Unsloth 2025.10.1: Fast Qwen3 patching. Transformers: 4.55.4. vLLM: 0.11.0.
   \\   /|    NVIDIA L40S. Num GPUs = 1. Max memory: 44.403 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-4B-Base with actual GPU utilization = 89.06%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM =

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

INFO 10-08 23:39:35 [backends.py:218] Compiling a graph for dynamic shape takes 32.11 s
INFO 10-08 23:40:11 [monitor.py:34] torch.compile takes 41.44 s in total
INFO 10-08 23:40:13 [gpu_worker.py:298] Available KV cache memory: 30.12 GiB
INFO 10-08 23:40:14 [kv_cache_utils.py:1087] GPU KV cache size: 219,344 tokens
INFO 10-08 23:40:14 [kv_cache_utils.py:1091] Maximum concurrency for 1,024 tokens per request: 214.20x
INFO 10-08 23:40:14 [vllm_utils.py:694] Unsloth: Running patched vLLM v1 `capture_model`.
WARNING 10-08 23:40:14 [gpu_model_runner.py:3643] CUDAGraphMode.FULL is not supported with FlashAttentionMetadataBuilder backend (support: AttentionCGSupport.UNIFORM_BATCH); setting cudagraph_mode=FULL_AND_PIECEWISE


Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:14<00:00,  4.64it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 43/43 [00:04<00:00,  9.20it/s]

INFO 10-08 23:40:33 [gpu_model_runner.py:3480] Graph capturing finished in 19 secs, took 1.29 GiB
INFO 10-08 23:40:33 [vllm_utils.py:701] Unsloth: Patched vLLM v1 graph capture finished in 19 secs.


INFO 10-08 23:40:34 [core.py:210] init engine (profile, create kv cache, warmup model) took 103.43 seconds
INFO 10-08 23:40:35 [llm.py:306] Supported_tasks: ('generate',)
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'post_feedforward_layernorm', 'norm2', 'attention_norm', 'q_norm', 'post_attention_layernorm', 'ffn_norm', 'k_norm', 'post_layernorm', 'input_layernorm', 'layer_norm1', 'layer_norm2', 'norm1']
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'post_feedforward_layernorm', 'norm2', 'attention_norm', 'q_norm', 'post_attention_layernorm', 'ffn_norm', 'k_norm', 'post_layernorm', 'cross_attn_input_layernorm', 'cross_attn_post_attention_layernorm', 'input_layernorm', 'layer_norm1', 'layer_norm2', 'norm1']


Unsloth 2025.10.1 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [5]:
# ============================================================================
# STEP 2: Define Custom Tags and System Prompt
# ============================================================================
reasoning_start = "<start_working_out>"
reasoning_end   = "<end_working_out>"
solution_start  = "<SOLUTION>"
solution_end    = "</SOLUTION>"

system_prompt = f"""You are an industrial maintenance expert analyzing machine sensor data.
Given sensor readings from a machine, analyze the data carefully to predict potential failures.
Place your reasoning between {reasoning_start} and {reasoning_end}.
Then, provide your prediction between {solution_start}{solution_end}
The prediction must be either: failure or normal"""

# ============================================================================
# STEP 3: Create Chat Template
# ============================================================================
chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"\
    "{% endif %}"

chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")\
    .replace("'{reasoning_start}'", f"'{reasoning_start}'")
tokenizer.chat_template = chat_template

In [6]:
# ============================================================================
# STEP 4: Load and Prepare Machine Sensor Dataset
# ============================================================================
# Load your CSV file
# Replace 'machine_sensor_data.csv' with your actual file path

# Download latest version
csv_file = '/workspace/data.csv'

print("Path to dataset files:", csv_file)

machine_df = pd.read_csv(csv_file)

# Ensure column names match (handle any case/space differences)
machine_df.columns = machine_df.columns.str.strip().str.lower()

# Map fail column to descriptive names
machine_df['status'] = machine_df['fail'].map({
    0: 'normal',
    1: 'failure'
})

print(f"Machine dataset shape: {machine_df.shape}")
print(f"\nClass distribution:")
print(machine_df['status'].value_counts())
print(f"\nFirst few rows:")
print(machine_df.head())

# Check for missing values
print(f"\nMissing values:")
print(machine_df.isnull().sum())

# Handle missing values if any
machine_df = machine_df.dropna()

Path to dataset files: /workspace/data.csv
Machine dataset shape: (944, 11)

Class distribution:
status
normal     551
failure    393
Name: count, dtype: int64

First few rows:
   footfall  tempmode  aq  uss  cs  voc  rp  ip  temperature  fail   status
0         0         7   7    1   6    6  36   3            1     1  failure
1       190         1   3    3   5    1  20   4            1     0   normal
2        31         7   2    2   6    1  24   6            1     0   normal
3        83         4   3    4   5    1  28   6            1     0   normal
4       640         7   5    6   4    0  68   6            1     0   normal

Missing values:
footfall       0
tempmode       0
aq             0
uss            0
cs             0
voc            0
rp             0
ip             0
temperature    0
fail           0
status         0
dtype: int64


In [7]:
# ============================================================================
# STEP 5: Create Reasoning Examples for Pre-training
# ============================================================================
def create_reasoning_example(row):
    """Create a reasoning example for each machine reading"""
    status = row['status']
    
    # Analyze different sensor patterns
    temp_status = 'high' if row['temperature'] > 75 else 'normal' if row['temperature'] > 50 else 'low'
    cs_status = 'high' if row['cs'] > 15 else 'normal' if row['cs'] > 5 else 'low'
    aq_status = 'poor' if row['aq'] < 50 else 'moderate' if row['aq'] < 150 else 'good'
    voc_status = 'high' if row['voc'] > 500 else 'moderate' if row['voc'] > 200 else 'low'
    
    reasoning_templates = [
        f"Analyzing sensor readings:\n"
        f"• Temperature: {row['temperature']}°C ({temp_status})\n"
        f"• Current Sensor: {row['cs']} A ({cs_status})\n"
        f"• Air Quality: {row['aq']} AQI ({aq_status})\n"
        f"• VOC Level: {row['voc']} ppb ({voc_status})\n"
        f"• Rotational Position: {row['rp']} RPM\n"
        f"• Input Pressure: {row['ip']} PSI\n\n"
        f"{'Critical indicators detected: High temperature and current draw suggest overheating. This points to potential failure.' if status == 'failure' else 'All parameters within normal operating ranges. Machine functioning properly.'}\n"
        f"Prediction: {status}",
        
        f"Machine diagnostics:\n"
        f"Temperature reading of {row['temperature']}°C is {temp_status}. "
        f"Current draw at {row['cs']}A is {cs_status}. "
        f"{'These elevated readings combined with VOC level of ' + str(row['voc']) + ' ppb suggest abnormal operation.' if status == 'failure' else 'Stable current and temperature indicate normal operation.'}\n\n"
        f"Ultrasonic sensor shows {row['uss']} units. "
        f"{'Proximity readings are unusual, possible misalignment.' if row['uss'] > 100 else 'Position detection is normal.'}\n\n"
        f"Based on the sensor pattern analysis, machine status is: {status}",
        
        f"Reviewing all sensors:\n"
        f"Footfall: {row['footfall']} | Temp Mode: {row['tempmode']}\n"
        f"Air Quality: {row['aq']} | USS: {row['uss']}\n"
        f"Current: {row['cs']}A | VOC: {row['voc']} ppb\n"
        f"RPM: {row['rp']} | Pressure: {row['ip']} PSI\n"
        f"Temperature: {row['temperature']}°C\n\n"
        f"{'Warning signs: Multiple parameters outside normal ranges. Temperature and current particularly concerning.' if status == 'failure' else 'System parameters stable. No anomalies detected.'}\n"
        f"Status: {status}",
    ]
    
    reasoning = np.random.choice(reasoning_templates)
    return reasoning

# Create balanced pre-training dataset
# Take equal samples from each class
n_samples_per_class = 15
pretrain_df = pd.concat([
    machine_df[machine_df['status'] == 'normal'].sample(n=min(n_samples_per_class, len(machine_df[machine_df['status'] == 'normal'])), random_state=42),
    machine_df[machine_df['status'] == 'failure'].sample(n=min(n_samples_per_class, len(machine_df[machine_df['status'] == 'failure'])), random_state=42)
]).reset_index(drop=True)

print(f"\nPre-training dataset class distribution:")
print(pretrain_df['status'].value_counts())

pretrain_df['reasoning'] = pretrain_df.apply(create_reasoning_example, axis=1)

def format_pretrain_dataset(row):
    problem = f"""Predict machine failure based on sensor data:
Footfall: {row['footfall']}
Temperature Mode: {row['tempmode']}
Air Quality: {row['aq']} AQI
Ultrasonic Sensor: {row['uss']}
Current Sensor: {row['cs']} A
VOC Level: {row['voc']} ppb
Rotational Position: {row['rp']} RPM
Input Pressure: {row['ip']} PSI
Temperature: {row['temperature']}°C"""
    
    final_prompt = (
        reasoning_start + row['reasoning'] + reasoning_end +
        solution_start + row['status'] + solution_end
    )
    
    return {
        "Messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": problem},
            {"role": "assistant", "content": final_prompt},
        ]
    }

pretrain_dataset = pretrain_df.apply(format_pretrain_dataset, axis=1).tolist()
pretrain_dataset = pd.DataFrame(pretrain_dataset)

# Tokenize
pretrain_dataset["text"] = tokenizer.apply_chat_template(
    pretrain_dataset["Messages"].values.tolist(), 
    tokenize=False
)
pretrain_dataset = Dataset.from_pandas(pretrain_dataset)

print(f"\nPre-training dataset size: {len(pretrain_dataset)}")
print("\nExample formatted message:")
print(tokenizer.apply_chat_template(pretrain_dataset[0]["Messages"], tokenize=False))



Pre-training dataset class distribution:
status
normal     15
failure    15
Name: count, dtype: int64

Pre-training dataset size: 30

Example formatted message:
You are an industrial maintenance expert analyzing machine sensor data.
Given sensor readings from a machine, analyze the data carefully to predict potential failures.
Place your reasoning between <start_working_out> and <end_working_out>.
Then, provide your prediction between <SOLUTION></SOLUTION>
The prediction must be either: failure or normal<|endoftext|>Predict machine failure based on sensor data:
Footfall: 0
Temperature Mode: 0
Air Quality: 3 AQI
Ultrasonic Sensor: 4
Current Sensor: 4 A
VOC Level: 0 ppb
Rotational Position: 48 RPM
Input Pressure: 6 PSI
Temperature: 24°C<start_working_out>Analyzing sensor readings:
• Temperature: 24°C (low)
• Current Sensor: 4 A (low)
• Air Quality: 3 AQI (poor)
• VOC Level: 0 ppb (low)
• Rotational Position: 48 RPM
• Input Pressure: 6 PSI

All parameters within normal operating ranges. 

In [8]:
# ============================================================================
# STEP 6: Pre-Fine-Tune for Format
# ============================================================================
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=pretrain_dataset,
    args=SFTConfig(
        dataset_text_field="text",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        warmup_steps=5,
        num_train_epochs=3,
        learning_rate=2e-4,
        logging_steps=5,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

print("\n" + "="*50)
print("Starting pre-training...")
print("="*50)
trainer.train()

# Test the model
text = tokenizer.apply_chat_template(
    pretrain_dataset[0]["Messages"][:2],
    tokenize=False,
    add_generation_prompt=True,
)

from transformers import TextStreamer
print("\n" + "="*50)
print("Testing pre-trained model:")
print("="*50)
_ = model.generate(
    **tokenizer(text, return_tensors="pt").to("cuda"),
    temperature=0,
    max_new_tokens=512,
    streamer=TextStreamer(tokenizer, skip_prompt=False),
)

# Clean up
del pretrain_dataset
torch.cuda.empty_cache()
import gc
gc.collect()

num_proc must be <= 30. Reducing num_proc to 30 for dataset of size 30.


Unsloth: Tokenizing ["text"] (num_proc=30):   0%|          | 0/30 [00:00<?, ? examples/s]


Starting pre-training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 30 | Num Epochs = 3 | Total steps = 45
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)


Step,Training Loss
5,2.122900
10,0.804600
15,0.261900
20,0.177700
25,0.143700
30,0.130700
35,0.112900
40,0.117400
45,0.113800


Unsloth: Will smartly offload gradients to save VRAM!

Testing pre-trained model:
You are an industrial maintenance expert analyzing machine sensor data.
Given sensor readings from a machine, analyze the data carefully to predict potential failures.
Place your reasoning between <start_working_out> and <end_working_out>.
Then, provide your prediction between <SOLUTION></SOLUTION>
The prediction must be either: failure or normal<|endoftext|>Predict machine failure based on sensor data:
Footfall: 0
Temperature Mode: 0
Air Quality: 3 AQI
Ultrasonic Sensor: 4
Current Sensor: 4 A
VOC Level: 0 ppb
Rotational Position: 48 RPM
Input Pressure: 6 PSI
Temperature: 24°C<start_working_out>Reviewing all sensors:
Footfall: 0 | Temp Mode: 0
Air Quality: 3 | USS: 4
Current: 4A | VOC: 0 ppb
RPM: 48 | Pressure: 6 PSI
Temperature: 24°C

System parameters stable. No anomalies detected.
Status: normal<end_working_out><SOLUTION>normal</SOLUTION><|endoftext|>


2266

In [9]:
# ============================================================================
# STEP 7: Prepare GRPO Dataset
# ============================================================================
# Use remaining samples for GRPO training
grpo_df = machine_df[~machine_df.index.isin(pretrain_df.index)].copy()

# Balance the GRPO dataset if needed
failure_samples = grpo_df[grpo_df['status'] == 'failure']
normal_samples = grpo_df[grpo_df['status'] == 'normal']

# Use all failure samples and sample normal to match (or vice versa)
n_failure = len(failure_samples)
n_normal = len(normal_samples)

if n_failure < n_normal:
    normal_samples = normal_samples.sample(n=n_failure, random_state=42)
else:
    failure_samples = failure_samples.sample(n=n_normal, random_state=42)

grpo_df = pd.concat([failure_samples, normal_samples]).reset_index(drop=True)
grpo_df = grpo_df.sample(frac=1, random_state=42).reset_index(drop=True)  # Shuffle

print(f"\nGRPO dataset class distribution:")
print(grpo_df['status'].value_counts())

def format_grpo_dataset(row):
    problem = f"""Predict machine failure based on sensor data:
Footfall: {row['footfall']}
Temperature Mode: {row['tempmode']}
Air Quality: {row['aq']} AQI
Ultrasonic Sensor: {row['uss']}
Current Sensor: {row['cs']} A
VOC Level: {row['voc']} ppb
Rotational Position: {row['rp']} RPM
Input Pressure: {row['ip']} PSI
Temperature: {row['temperature']}°C"""
    
    return {
        "prompt": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": problem},
        ],
        "answer": row['status']
    }

grpo_dataset = grpo_df.apply(format_grpo_dataset, axis=1).tolist()
grpo_dataset = pd.DataFrame(grpo_dataset)
grpo_dataset = Dataset.from_pandas(grpo_dataset)

print(f"\nGRPO dataset size: {len(grpo_dataset)}")
print(f"First example: {grpo_dataset[0]}")



GRPO dataset class distribution:
status
normal     389
failure    389
Name: count, dtype: int64

GRPO dataset size: 778
First example: {'prompt': [{'content': 'You are an industrial maintenance expert analyzing machine sensor data.\nGiven sensor readings from a machine, analyze the data carefully to predict potential failures.\nPlace your reasoning between <start_working_out> and <end_working_out>.\nThen, provide your prediction between <SOLUTION></SOLUTION>\nThe prediction must be either: failure or normal', 'role': 'system'}, {'content': 'Predict machine failure based on sensor data:\nFootfall: 0\nTemperature Mode: 1\nAir Quality: 3 AQI\nUltrasonic Sensor: 2\nCurrent Sensor: 6 A\nVOC Level: 1 ppb\nRotational Position: 34 RPM\nInput Pressure: 6 PSI\nTemperature: 22°C', 'role': 'user'}], 'answer': 'normal'}


In [15]:
# ============================================================================
# STEP 8: Define Reward Functions (IMPROVED)
# ============================================================================
solution_end_regex = r"</SOLUTION>[\s]{0,}" + \
    "(?:" + re.escape(tokenizer.eos_token) + ")?"

match_format = re.compile(
    rf"{reasoning_end}.*?"
    rf"{solution_start}(.+?){solution_end_regex}"
    rf"[\s]{{0,}}$",
    flags=re.MULTILINE | re.DOTALL
)

def match_format_exactly(completions, **kwargs):
    """Reward correct formatting"""
    scores = []
    for completion in completions:
        response = completion[0]["content"]
        score = 2.0 if match_format.search(response) is not None else -1.0  # Reduced from 3.0
        scores.append(score)
    return scores

def match_format_approximately(completions, **kwargs):
    """Reward partial formatting"""
    scores = []
    for completion in completions:
        response = completion[0]["content"]
        score = 0
        score += 0.3 if response.count(reasoning_end) == 1 else -0.5
        score += 0.3 if response.count(solution_start) == 1 else -0.5
        score += 0.3 if response.count(solution_end) == 1 else -0.5
        scores.append(score)
    return scores

def check_answer(prompts, completions, answer, **kwargs):
    """Reward correct failure prediction with class balancing"""
    responses = [completion[0]["content"] for completion in completions]
    
    extracted_responses = [
        guess.group(1).strip().lower()
        if (guess := match_format.search(r)) is not None else None
        for r in responses
    ]
    
    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(-3.0)
            continue
        
        true_answer = true_answer.strip().lower()
        
        # Exact match - HIGHER rewards for correct predictions
        if guess == true_answer:
            # Give EXTRA reward for correctly predicting 'normal' since model is biased toward 'failure'
            if true_answer == 'normal':
                scores.append(8.0)  # Increased reward for normal predictions
            else:
                scores.append(6.0)  # Standard reward for failure predictions
        # Partial match
        elif true_answer in guess or guess in true_answer:
            scores.append(2.0)
        # STRONG penalty for wrong predictions
        else:
            # Extra penalty if predicting failure when it's normal (to combat bias)
            if guess == 'failure' and true_answer == 'normal':
                scores.append(-6.0)  # Strong penalty for false alarms
            else:
                scores.append(-4.0)  # Standard penalty for other mistakes
    
    return scores

def diversity_reward(prompts, completions, answer, **kwargs):
    """Encourage diverse predictions to prevent always predicting one class"""
    responses = [completion[0]["content"] for completion in completions]
    
    extracted_responses = [
        guess.group(1).strip().lower()
        if (guess := match_format.search(r)) is not None else None
        for r in responses
    ]
    
    # Count predictions
    failure_count = sum(1 for r in extracted_responses if r and 'failure' in r)
    normal_count = sum(1 for r in extracted_responses if r and 'normal' in r)
    
    # Penalize if all predictions are the same
    scores = []
    total_valid = failure_count + normal_count
    
    if total_valid > 0:
        # If all predictions are the same class, give penalty
        if failure_count == total_valid or normal_count == total_valid:
            # All predictions are same - penalty
            scores = [-1.0] * len(completions)
        else:
            # Good diversity - small reward
            scores = [0.5] * len(completions)
    else:
        scores = [0] * len(completions)
    
    return scores

global PRINTED_TIMES
PRINTED_TIMES = 0
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 5

def print_examples(prompts, completions, answer, **kwargs):
    """Print examples periodically"""
    global PRINTED_TIMES, PRINT_EVERY_STEPS
    
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        question = prompts[0][-1]["content"]
        response = completions[0][0]["content"]
        
        extracted = None
        if (match := match_format.search(response)):
            extracted = match.group(1).strip()
        
        # Count all predictions in this batch
        all_predictions = []
        for completion in completions:
            resp = completion[0]["content"]
            if (m := match_format.search(resp)):
                all_predictions.append(m.group(1).strip().lower())
        
        failure_count = sum(1 for p in all_predictions if 'failure' in p)
        normal_count = sum(1 for p in all_predictions if 'normal' in p)
        
        print("\n" + "="*60)
        print(f"STEP {PRINTED_TIMES}")
        print("="*60)
        print(f"Batch Prediction Distribution: Normal={normal_count}, Failure={failure_count}")
        print(f"\nQuestion:\n{question}")
        print(f"\nTrue Status: {answer[0]}")
        print(f"\nModel Response:\n{response}")
        print(f"\nExtracted Prediction: {extracted}")
        print("="*60)
    
    PRINTED_TIMES += 1
    return [0] * len(completions)  # Neutral reward

In [16]:
# ============================================================================
# STEP 9: GRPO Training (IMPROVED)
# ============================================================================
from vllm import SamplingParams
from trl import GRPOConfig, GRPOTrainer

max_prompt_length = 512
max_completion_length = max_seq_length - max_prompt_length

vllm_sampling_params = SamplingParams(
    min_p=0.1,
    top_p=0.95,  # Reduced from 1.0 to encourage more diversity
    top_k=-1,
    temperature=1.2,  # Increased temperature for more exploration
    seed=3407,
    stop=[tokenizer.eos_token],
    include_stop_str_in_output=True,
)

training_args = GRPOConfig(
    vllm_sampling_params=vllm_sampling_params,
    temperature=1.2,  # Increased for exploration
    learning_rate=3e-6,  # Reduced learning rate for more stable training
    weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    optim="adamw_8bit",
    logging_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,  # Increased for more stable gradients
    num_generations=8,  # Increased generations for better exploration
    max_prompt_length=max_prompt_length,
    max_completion_length=max_completion_length,
    num_train_epochs=3,  # Increased epochs
    save_steps=50,
    output_dir="outputs",
    report_to="none",
)

grpo_trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    args=training_args,
    train_dataset=grpo_dataset,
    reward_funcs=[
        match_format_exactly,
        match_format_approximately,
        check_answer,
        diversity_reward,  # NEW: Add diversity reward
        print_examples,
    ],
)

print("\n" + "="*50)
print("Starting GRPO training...")
print("="*50)
grpo_trainer.train()

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8

Starting GRPO training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 778 | Num Epochs = 3 | Total steps = 582
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)



STEP 0
Batch Prediction Distribution: Normal=22, Failure=10

Question:
Predict machine failure based on sensor data:
Footfall: 27
Temperature Mode: 7
Air Quality: 3 AQI
Ultrasonic Sensor: 7
Current Sensor: 4 A
VOC Level: 3 ppb
Rotational Position: 63 RPM
Input Pressure: 2 PSI
Temperature: 19°C

True Status: failure

Model Response:
Machine diagnostics:
Temperature: 19°C (low)
Current Draw: 4 A (low)
Ultrasonic: 7 (poor)
VOC Level: 3 ppb (low)
Air Quality: 3 AQI (poor)

Steady operation with low readings on multiple parameters. Diagnostics indicate normal functioning.
Machine Status: normal<end_working_out><SOLUTION>normal</SOLUTION>

Extracted Prediction: normal


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_approximately / mean,rewards / match_format_approximately / std,rewards / check_answer / mean,rewards / check_answer / std,rewards / diversity_reward / mean,rewards / diversity_reward / std,rewards / print_examples / mean,rewards / print_examples / std
1,0.005700,4.775000,5.651797,91.406250,52.000000,157.000000,0.000000,91.406250,52.000000,157.000000,1.415258,2.000000,0.000000,0.900000,0.000000,1.375000,6.168337,0.500000,0.000000,0.000000,0.000000
2,0.007100,3.887500,6.646852,103.968750,58.000000,512.000000,0.031250,90.806450,58.000000,164.000000,1.775805,1.906250,0.530330,0.825000,0.424264,0.656250,7.028098,0.500000,0.000000,0.000000,0.000000
3,0.005300,6.150000,3.650856,88.750000,57.000000,144.000000,0.000000,88.750000,57.000000,144.000000,1.319754,2.000000,0.000000,0.900000,0.000000,2.750000,5.370168,0.500000,0.000000,0.000000,0.000000
4,0.005000,5.712500,6.019353,89.187500,58.000000,136.000000,0.000000,89.187500,58.000000,136.000000,1.256120,2.000000,0.000000,0.900000,0.000000,2.312500,6.034511,0.500000,0.000000,0.000000,0.000000
5,0.004600,3.150000,4.008919,88.406250,61.000000,137.000000,0.000000,88.406250,61.000000,137.000000,1.151198,2.000000,0.000000,0.900000,0.000000,-0.250000,4.918694,0.500000,0.000000,0.000000,0.000000
6,0.005500,4.087500,5.837314,93.031250,58.000000,140.000000,0.000000,93.031250,58.000000,140.000000,1.371863,2.000000,0.000000,0.900000,0.000000,0.687500,5.647937,0.500000,0.000000,0.000000,0.000000
7,0.004600,4.337500,5.501804,90.812500,53.000000,158.000000,0.000000,90.812500,53.000000,158.000000,1.157751,2.000000,0.000000,0.900000,0.000000,0.937500,5.656497,0.500000,0.000000,0.000000,0.000000
8,0.004500,5.275000,6.078760,93.062500,58.000000,133.000000,0.000000,93.062500,58.000000,133.000000,1.129865,2.000000,0.000000,0.900000,0.000000,1.875000,6.115818,0.500000,0.000000,0.000000,0.000000
9,0.007400,4.750000,6.750526,89.656250,58.000000,162.000000,0.000000,89.656250,58.000000,162.000000,1.857189,1.906250,0.530330,0.875000,0.141421,1.468750,6.623440,0.500000,0.000000,0.000000,0.000000
10,0.004900,3.900000,5.598876,96.500000,46.000000,170.000000,0.000000,96.500000,46.000000,170.000000,1.231735,2.000000,0.000000,0.900000,0.000000,0.500000,5.633998,0.500000,0.000000,0.000000,0.000000



STEP 5
Batch Prediction Distribution: Normal=17, Failure=15

Question:
Predict machine failure based on sensor data:
Footfall: 8
Temperature Mode: 0
Air Quality: 5 AQI
Ultrasonic Sensor: 1
Current Sensor: 6 A
VOC Level: 4 ppb
Rotational Position: 29 RPM
Input Pressure: 3 PSI
Temperature: 20°C

True Status: failure

Model Response:
Machine diagnostics:
Temperature reading of 20°C is low compared to normal operation. This is concerning. High current draw of 6A suggests elevated power consumption, indicating increased load. These elevated readings combined with VOC level of 4 ppb and unstable position of 29 RPM suggest abnormal operation.

Based on sensor patterns and machine conditions, machine operation is normal.
Prediction: normal<end_working_out><SOLUTION>failure</SOLUTION>

Extracted Prediction: failure

STEP 10
Batch Prediction Distribution: Normal=21, Failure=11

Question:
Predict machine failure based on sensor data:
Footfall: 0
Temperature Mode: 4
Air Quality: 3 AQI
Ultrasonic 

TrainOutput(global_step=582, training_loss=0.0051602536213945895, metrics={'train_runtime': 7178.306, 'train_samples_per_second': 0.325, 'train_steps_per_second': 0.081, 'total_flos': 0.0, 'train_loss': 0.0051602536213945895})

In [17]:
# ============================================================================
# STEP 10: Test the Model
# ============================================================================
print("\n" + "="*50)
print("Testing final model on new examples:")
print("="*50)

# Use some held-out test examples
test_df = machine_df[~machine_df.index.isin(pretrain_df.index) & 
                      ~machine_df.index.isin(grpo_df.index)].head(3)

if len(test_df) == 0:
    # If no separate test set, use some from GRPO
    test_df = grpo_df.sample(n=3, random_state=123)

for i, (idx, row) in enumerate(test_df.iterrows()):
    problem = f"""Predict machine failure based on sensor data:
Footfall: {row['footfall']}
Temperature Mode: {row['tempmode']}
Air Quality: {row['aq']} AQI
Ultrasonic Sensor: {row['uss']}
Current Sensor: {row['cs']} A
VOC Level: {row['voc']} ppb
Rotational Position: {row['rp']} RPM
Input Pressure: {row['ip']} PSI
Temperature: {row['temperature']}°C"""
    
    text = tokenizer.apply_chat_template(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": problem},
        ],
        tokenize=False,
        add_generation_prompt=True,
    )
    
    print(f"\n{'='*60}")
    print(f"Test Example {i+1} (Actual: {row['status']})")
    print('='*60)
    _ = model.generate(
        **tokenizer(text, return_tensors="pt").to("cuda"),
        temperature=0.3,
        max_new_tokens=512,
        streamer=TextStreamer(tokenizer, skip_prompt=True),
    )



Testing final model on new examples:

Test Example 1 (Actual: normal)
Machine diagnostics:
Temperature reading of 22°C is low. Current draw at 6A is low. Stable current and temperature indicate normal operation.

Ultrasonic sensor shows 3 units. Position detection is normal.

Based on the sensor pattern analysis, machine is functioning properly.
Prediction: normal<end_working_out><SOLUTION>normal</SOLUTION><|endoftext|>

Test Example 2 (Actual: failure)
Machine diagnostics:
Temperature reading of 22°C is low. Current draw at 6A is low. These elevated readings combined with VOC level of 6 ppb suggest abnormal operation.

Ultrasonic sensor shows 1 units. Position detection is normal.

Based on the sensor pattern analysis, machine is functioning abnormally.
Prediction: failure<end_working_out><SOLUTION>failure</SOLUTION><|endoftext|>

Test Example 3 (Actual: normal)
Machine diagnostics:
Temperature reading of 22°C is low. Current draw at 6A is normal. Stable current and temperature indic

In [18]:
# ============================================================================
# STEP 10.5: Quick Calibration (Add BEFORE Step 11)
# ============================================================================
print("\n" + "="*50)
print("Applying post-training calibration...")
print("="*50)

# Create a small calibration set
calibration_df = grpo_df.sample(n=min(50, len(grpo_df)), random_state=42)

def get_raw_logits(row, model, tokenizer):
    """Get model's confidence for each class"""
    problem = f"""Predict machine failure based on sensor data:
Footfall: {row['footfall']}
Temperature Mode: {row['tempmode']}
Air Quality: {row['aq']} AQI
Ultrasonic Sensor: {row['uss']}
Current Sensor: {row['cs']} A
VOC Level: {row['voc']} ppb
Rotational Position: {row['rp']} RPM
Input Pressure: {row['ip']} PSI
Temperature: {row['temperature']}°C"""
    
    text = tokenizer.apply_chat_template(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": problem},
        ],
        tokenize=False,
        add_generation_prompt=True,
    )
    
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    
    # Get model output with logits
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            temperature=0.7,
            max_new_tokens=512,
            output_scores=True,
            return_dict_in_generate=True,
        )
    
    response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=False)
    match = match_format.search(response)
    
    if match:
        prediction = match.group(1).strip().lower()
        if 'failure' in prediction:
            return 'failure'
        elif 'normal' in prediction:
            return 'normal'
    
    return 'failure'  # Default bias

# Calculate bias on calibration set
calibration_predictions = []
for idx, row in calibration_df.iterrows():
    pred = get_raw_logits(row, model, tokenizer)
    calibration_predictions.append(pred)

failure_rate = sum(1 for p in calibration_predictions if p == 'failure') / len(calibration_predictions)
print(f"\nModel failure prediction rate on calibration set: {failure_rate*100:.1f}%")
print(f"Actual failure rate in calibration set: {(calibration_df['status'] == 'failure').mean()*100:.1f}%")

if failure_rate > 0.7:
    print("\n⚠️  WARNING: Model is heavily biased toward predicting 'failure'")
    print("Recommendation: Retrain with the improved reward functions!")
elif failure_rate < 0.3:
    print("\n⚠️  WARNING: Model is heavily biased toward predicting 'normal'")
    print("Recommendation: Retrain with the improved reward functions!")
else:
    print("\n✓ Model predictions appear reasonably balanced")


Applying post-training calibration...

Model failure prediction rate on calibration set: 100.0%
Actual failure rate in calibration set: 56.0%

⚠️  WARNING: Model is heavily biased toward predicting 'failure'
Recommendation: Retrain with the improved reward functions!


In [19]:
# ============================================================================
# STEP 11: Evaluate Model on All Data and Calculate Accuracy
# ============================================================================
print("\n" + "="*50)
print("Evaluating model on entire dataset...")
print("="*50)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import time

# Load all data
all_data_df = machine_df.copy()

def predict_machine_status(row, model, tokenizer):
    """Make a prediction for a single data point"""
    problem = f"""Predict machine failure based on sensor data:
Footfall: {row['footfall']}
Temperature Mode: {row['tempmode']}
Air Quality: {row['aq']} AQI
Ultrasonic Sensor: {row['uss']}
Current Sensor: {row['cs']} A
VOC Level: {row['voc']} ppb
Rotational Position: {row['rp']} RPM
Input Pressure: {row['ip']} PSI
Temperature: {row['temperature']}°C"""
    
    text = tokenizer.apply_chat_template(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": problem},
        ],
        tokenize=False,
        add_generation_prompt=True,
    )
    
    # Generate prediction
    outputs = model.generate(
        **tokenizer(text, return_tensors="pt").to("cuda"),
        temperature=0.1,  # Low temperature for more consistent predictions
        max_new_tokens=512,
        do_sample=False,  # Greedy decoding for evaluation
    )
    
    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    
    # Extract the prediction using regex
    match = match_format.search(response)
    if match:
        prediction = match.group(1).strip().lower()
        # Map any variations to standard labels
        if 'failure' in prediction or 'fail' in prediction:
            return 'failure'
        elif 'normal' in prediction or 'ok' in prediction or 'healthy' in prediction:
            return 'normal'
    
    return None  # Failed to extract prediction

# Make predictions for all data
print(f"\nMaking predictions on {len(all_data_df)} samples...")
print("This may take a few minutes...\n")

predictions = []
true_labels = []
failed_predictions = 0

start_time = time.time()

for idx, row in all_data_df.iterrows():
    if idx % 10 == 0:
        print(f"Progress: {idx}/{len(all_data_df)} samples processed...", end='\r')
    
    prediction = predict_machine_status(row, model, tokenizer)
    
    if prediction is None:
        failed_predictions += 1
        # Default to 'normal' for failed predictions
        prediction = 'normal'
    
    predictions.append(prediction)
    true_labels.append(row['status'])

end_time = time.time()
print(f"\n✓ Completed {len(all_data_df)} predictions in {end_time - start_time:.2f} seconds")
print(f"  Average time per prediction: {(end_time - start_time) / len(all_data_df):.3f} seconds")

if failed_predictions > 0:
    print(f"\n⚠ Warning: {failed_predictions} predictions failed to extract properly (defaulted to 'normal')")

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, pos_label='failure', zero_division=0)
recall = recall_score(true_labels, predictions, pos_label='failure', zero_division=0)
f1 = f1_score(true_labels, predictions, pos_label='failure', zero_division=0)

# Confusion matrix
cm = confusion_matrix(true_labels, predictions, labels=['normal', 'failure'])

print("\n" + "="*60)
print("EVALUATION RESULTS")
print("="*60)
print(f"\nOverall Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precision (Failure): {precision:.4f}")
print(f"Recall (Failure): {recall:.4f}")
print(f"F1-Score (Failure): {f1:.4f}")

print("\nConfusion Matrix:")
print("                 Predicted")
print("                Normal  Failure")
print(f"Actual Normal    {cm[0][0]:6d}  {cm[0][1]:7d}")
print(f"       Failure   {cm[1][0]:6d}  {cm[1][1]:7d}")

print("\nDetailed Classification Report:")
print(classification_report(true_labels, predictions, target_names=['normal', 'failure']))

# Class-wise accuracy
normal_correct = cm[0][0]
normal_total = cm[0][0] + cm[0][1]
failure_correct = cm[1][1]
failure_total = cm[1][0] + cm[1][1]

print(f"\nClass-wise Performance:")
print(f"  Normal class accuracy:  {normal_correct}/{normal_total} ({normal_correct/normal_total*100:.2f}%)")
print(f"  Failure class accuracy: {failure_correct}/{failure_total} ({failure_correct/failure_total*100:.2f}%)")

# Save results
results_df = pd.DataFrame({
    'true_label': true_labels,
    'predicted_label': predictions,
    'correct': [t == p for t, p in zip(true_labels, predictions)]
})

# Add original data
results_df = pd.concat([all_data_df.reset_index(drop=True), results_df], axis=1)
results_df.to_csv('model_evaluation_results.csv', index=False)
print(f"\n✓ Detailed results saved to 'model_evaluation_results.csv'")

# Show some example predictions
print("\n" + "="*60)
print("SAMPLE PREDICTIONS")
print("="*60)

# Show correct predictions
print("\n✓ Correct Predictions (5 examples):")
correct_samples = results_df[results_df['correct'] == True].sample(n=min(5, len(results_df[results_df['correct'] == True])), random_state=42)
for idx, row in correct_samples.iterrows():
    print(f"\n  Actual: {row['true_label']:8s} | Predicted: {row['predicted_label']:8s}")
    print(f"  Sensors: Temp={row['temperature']}°C, Current={row['cs']}A, VOC={row['voc']}ppb")

# Show incorrect predictions
if len(results_df[results_df['correct'] == False]) > 0:
    print("\n✗ Incorrect Predictions (5 examples):")
    incorrect_samples = results_df[results_df['correct'] == False].sample(n=min(5, len(results_df[results_df['correct'] == False])), random_state=42)
    for idx, row in incorrect_samples.iterrows():
        print(f"\n  Actual: {row['true_label']:8s} | Predicted: {row['predicted_label']:8s}")
        print(f"  Sensors: Temp={row['temperature']}°C, Current={row['cs']}A, VOC={row['voc']}ppb")

print("\n" + "="*60)
print("EVALUATION COMPLETE")
print("="*60)


Evaluating model on entire dataset...

Making predictions on 944 samples...
This may take a few minutes...

Progress: 940/944 samples processed...
✓ Completed 944 predictions in 2689.55 seconds
  Average time per prediction: 2.849 seconds

EVALUATION RESULTS

Overall Accuracy: 0.4163 (41.63%)
Precision (Failure): 0.4163
Recall (Failure): 1.0000
F1-Score (Failure): 0.5879

Confusion Matrix:
                 Predicted
                Normal  Failure
Actual Normal         0      551
       Failure        0      393

Detailed Classification Report:
              precision    recall  f1-score   support

      normal       0.42      1.00      0.59       393
     failure       0.00      0.00      0.00       551

    accuracy                           0.42       944
   macro avg       0.21      0.50      0.29       944
weighted avg       0.17      0.42      0.24       944


Class-wise Performance:
  Normal class accuracy:  0/551 (0.00%)
  Failure class accuracy: 393/393 (100.00%)

✓ Detailed 

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [ ]:
# # ============================================================================
# # STEP 12: Save the Model (HuggingFace Format)
# # ============================================================================
# model.save_pretrained("machine_failure_model")
# tokenizer.save_pretrained("machine_failure_model")
# print("\n✓ Model saved to 'machine_failure_model'")

# # ============================================================================
# # STEP 13: Export to GGUF Format for LM Studio
# # ============================================================================
# print("\n" + "="*50)
# print("Exporting to GGUF format for LM Studio...")
# print("="*50)

# # Export 8-bit quantized version (recommended balance)
# print("\n[1/2] Exporting Q8_0 (8-bit quantization)...")
# model.save_pretrained_gguf(
#     "machine_failure_gguf_q8",
#     tokenizer,
#     quantization_method="q8_0"
# )
# print("✓ Q8_0 version saved to 'machine_failure_gguf_q8' folder")

# # Export 4-bit quantized version (smallest, fastest)
# print("\n[2/2] Exporting Q4_K_M (4-bit quantization)...")
# model.save_pretrained_gguf(
#     "machine_failure_gguf_q4",
#     tokenizer,
#     quantization_method="q4_k_m"
# )
# print("✓ Q4_K_M version saved to 'machine_failure_gguf_q4' folder")

# print("\n" + "="*50)
# print("✓ All GGUF exports completed!")
# print("="*50)
# print("\nGGUF files created:")
# print("  1. machine_failure_gguf_q8/   (8-bit - recommended)")
# print("  2. machine_failure_gguf_q4/   (4-bit - smallest/fastest)")